In [2]:
%load_ext autoreload
%autoreload 2                                                                                                                          

In [3]:
import configparser
import os.path
from os import path
from importlib import reload
import wandb


creds_path_ar = ["../../credentials.ini","credentials.colab.ini"]
PATH_ROOT = ""
PATH_DATA = ""

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        PATH_ROOT = config_parser['MAIN']["PATH_ROOT"]
        PATH_DATA = config_parser['MAIN']["PATH_DATA"]
        WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
        ENV = config_parser['MAIN']["ENV"]
        break

if ENV=="COLAB":
  from google.colab import drive
  mount_path = '/content/gdrive/'
  drive.mount(mount_path)



In [3]:
if WANDB_enable == True:
    wandb.init(project="sota-mafat-base")
    os.environ['WANDB_NOTEBOOK_NAME'] = '[SS]TCN_pytorch'

In [4]:
cd {PATH_ROOT}

/home/shaul/workspace/GitHub/sota-mafat-radar


In [5]:
import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from matplotlib.colors import LinearSegmentedColormap
from termcolor import colored

from src.data import feat_data, get_data, get_data_pipeline
from src.models import arch_setup, base_base_model, alex_model, tcn_model, tcn_model2

# Set seed for reproducibility of results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)


random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [17]:
config = dict()
config['num_tracks'] = 4
config['val_ratio'] = 6
config['shift_segment'] = list(np.floor(np.linspace(1,31,10)).astype(int))
config['get_shifts'] = True
config['get_horizontal_flip'] = True
config['get_vertical_flip'] = False

In [18]:
train_x, train_y, val_x, val_y = get_data_pipeline.pipeline_trainval(PATH_DATA, config)

In [19]:
train_set = arch_setup.DS(train_x.squeeze(),train_y)
val_set= arch_setup.DS(val_x.squeeze(),val_y)

train_loader=DataLoader(dataset= train_set, batch_size = 64, shuffle = True, num_workers = 2)
val_loader=DataLoader(dataset= val_set, batch_size = 64, shuffle = True, num_workers = 2)

In [20]:
import importlib

In [21]:
importlib.reload(tcn_model2)
importlib.reload(arch_setup)

<module 'src.models.arch_setup' from '/home/shaul/workspace/GitHub/sota-mafat-radar/src/models/arch_setup.py'>

In [22]:
model = tcn_model2.TCNModel([16,32,64,128], kernel_size=5, dropout= 0.2)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), betas=(0.9, 0.999), lr=1e-3, weight_decay=0)

model.to(device);

if WANDB_enable == True:
    wandb.init(project="sota-mafat-tcn")
    os.environ['WANDB_NOTEBOOK_NAME'] = '[SS]TCN_pytorch'
    wandb.watch(model)
    wandb.log(config)
    wandb.config['num_channels'] = "[16,32,64,128]"
    wandb.config['kernel_size'] = 5
    wandb.config['dropout'] = 0.2

In [23]:
log = arch_setup.train_epochs(train_loader,val_loader,model,criterion,optimizer,num_epochs= 25,device=device,train_y=train_y,val_y=val_y, WANDB_enable = WANDB_enable, wandb= wandb)

  7%|▋         | 117/1719 [00:21<05:07,  5.20it/s]

KeyboardInterrupt: 

In [ ]:
arch_setup.plot_loss_train_test(log,model)

In [ ]:
arch_setup.plot_ROC_local_gpu(train_loader,val_loader,model,device)

## Submission:

In [ ]:
from src.features import specto_feat

In [ ]:
test_path = 'MAFAT RADAR Challenge - Public Test Set V1'
test_df = get_data.load_data(PATH_ROOT+PATH_DATA+test_path)
test_df = specto_feat.data_preprocess(test_df.copy())
test_x = test_df['iq_sweep_burst']
test_x = test_x.reshape(list(test_x.shape)+[1])

In [ ]:
# Creating DataFrame with the probability prediction for each segment
submission =  pd.DataFrame()
submission['segment_id'] = test_df['segment_id']
submission['prediction'] = model(torch.from_numpy(test_x.squeeze()).type(torch.FloatTensor)).detach().numpy()
submission['prediction'] = submission['prediction'].astype('float')

# Save submission
submission.to_csv('submission.csv', index=False)

### NO MANDS LAND

In [25]:
dummy = np.random.rand(64,126,32)
dummy = torch.from_numpy(dummy).to(device,dtype=torch.float32)

In [26]:
model = tcn_model2.TemporalConvNet(1,num_channels=[32,1], kernel_size=3, dropout=0.2)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), betas=(0.9, 0.999), lr=1e-4, weight_decay=0)

In [27]:
y_hat = model(dummy)

In [28]:
y_hat

tensor([[0.4837],
        [0.4805],
        [0.4212],
        [0.4866],
        [0.3841],
        [0.4699],
        [0.4656],
        [0.4825],
        [0.5212],
        [0.5032],
        [0.4936],
        [0.5210],
        [0.5122],
        [0.4088],
        [0.4249],
        [0.4931],
        [0.4387],
        [0.4319],
        [0.4234],
        [0.5035],
        [0.4766],
        [0.4939],
        [0.4831],
        [0.5082],
        [0.5182],
        [0.4425],
        [0.4233],
        [0.4301],
        [0.4561],
        [0.4644],
        [0.5010],
        [0.4759],
        [0.4411],
        [0.5096],
        [0.5057],
        [0.4433],
        [0.4974],
        [0.4348],
        [0.4760],
        [0.5012],
        [0.4587],
        [0.4760],
        [0.4830],
        [0.4650],
        [0.4374],
        [0.5215],
        [0.4740],
        [0.5101],
        [0.4593],
        [0.4478],
        [0.4065],
        [0.4012],
        [0.3653],
        [0.5244],
        [0.4627],
        [0

In [42]:
def get_ndarray(embedding_values):
    results = []
    for row in embedding_values:
        arr = np.array(row)
        results.append(
            np.pad(arr, ((10 - arr.shape[0], 0), (0, 0)), 'constant')
        )
    # shape: (examples, emb_dim, seq_length)
    return np.transpose(np.stack(results), (0, 2, 1))

In [44]:
get_ndarray(df_train['audio_embedding']).shape

(1195, 128, 10)